In [82]:
import pretty_midi
import numpy as np

In [83]:
input_path = "transcribed_output/bass_output.mid"
output_path = "transcribed_output/bass_output_cleaned.mid"

In [84]:
# Load MIDI
midi = pretty_midi.PrettyMIDI("transcribed_output/bass_output.mid")

In [85]:
# Work on the first instrument (or find your bass track)
instrument = midi.instruments[0]

In [86]:
[note for note in instrument.notes]

[Note(start=0.000000, end=0.022727, pitch=51, velocity=100),
 Note(start=0.022727, end=0.604545, pitch=39, velocity=100),
 Note(start=0.604545, end=0.627273, pitch=31, velocity=100),
 Note(start=0.627273, end=0.672727, pitch=28, velocity=100),
 Note(start=0.672727, end=0.743182, pitch=27, velocity=100),
 Note(start=0.743182, end=0.765909, pitch=32, velocity=100),
 Note(start=0.765909, end=0.788636, pitch=40, velocity=100),
 Note(start=0.788636, end=0.859091, pitch=39, velocity=100),
 Note(start=0.859091, end=0.881818, pitch=29, velocity=100),
 Note(start=0.881818, end=0.904545, pitch=28, velocity=100),
 Note(start=0.904545, end=0.929545, pitch=27, velocity=100),
 Note(start=0.929545, end=0.952273, pitch=28, velocity=100),
 Note(start=0.952273, end=0.975000, pitch=27, velocity=100),
 Note(start=0.975000, end=0.997727, pitch=32, velocity=100),
 Note(start=0.997727, end=1.022727, pitch=51, velocity=100),
 Note(start=1.022727, end=1.277273, pitch=39, velocity=100),
 Note(start=1.277273, en

In [87]:
# ----------------------------
# Define helpers
# ----------------------------
def quantize_time(time, grid_size, pm):
    """Quantize a time value to the nearest grid point."""
    # Get beat grid from PrettyMIDI
    beats = pm.get_beats()
    if len(beats) < 2:
        return time  # no tempo info, skip

    # Grid spacing in seconds
    grid = np.arange(beats[0], beats[-1], grid_size)
    return grid[np.argmin(np.abs(grid - time))]

In [88]:
cleaned_notes = []
for note in instrument.notes:
    # # Keep only notes in 4-string bass range (E1=40 to G4=67 in MIDI numbers)
    # if 40 <= note.pitch <= 67:
    # Remove very short noise notes (< 60 ms)
    if note.end - note.start > 0.12:
        # Clamp velocities to a usable range
        note.velocity = min(max(note.velocity, 40), 100)

        # Quantize note start and end to nearest 1/16th note
        quantized_start = quantize_time(note.start, 0.25, midi)  # quarter-note subdivisions
        quantized_end = quantize_time(note.end, 0.25, midi)

        # Ensure quantized_end > quantized_start
        if quantized_end <= quantized_start:
            quantized_end = quantized_start + 0.1

        note.start = quantized_start
        note.end = quantized_end

        cleaned_notes.append(note)

In [89]:
# Replace notes with cleaned list
instrument.notes = cleaned_notes

In [90]:
# Save cleaned MIDI
midi.write(output_path)